In [1]:
# Database Connection  - Read Logs 
import pandas as pd
from clickhouse_driver import Client
client = Client(host='34.70.65.12', user = "default", password = "", database = "")

In [2]:
dogshit_query = """
SELECT
	geo_region,
	domain,
	toFloat64(sum(win_cost_micros_usd))/ 1000000 as cost,
	sum(clicks + companion_clicks) as clicks,
	count() as impressions,
	cost as CPC,
	today() - toDate(min(bid_time)) AS day_in_list
from
	Wins_buffer
where
	line_item_id in (113,109)
GROUP By
	geo_region,
	`domain`
having 
clicks = 0 and CPC > 0.3
order by
	CPC desc
"""

import pandas as pd
pd.set_option('display.max_columns', None)
result, columns = client.execute(dogshit_query, with_column_types=True)

voidu_dogshit = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

voidu_dogshit['value'] = 0.01 
voidu_dogshit.drop(columns=['cost','clicks','impressions','CPC','day_in_list'])

,geo_region,domain,value
0,USA/CA,www.shockwave.com,0.01
1,USA/CO,www1.flightrising.com,0.01
2,CAN/ON,www1.flightrising.com,0.01
3,USA/TX,www.shockwave.com,0.01
4,USA/PA,www1.flightrising.com,0.01
5,USA/MN,www1.flightrising.com,0.01
6,USA/NC,www1.flightrising.com,0.01
7,USA/NC,www.coolmathgames.com,0.01
8,USA/CA,na.op.gg,0.01
9,USA/WA,www.reddit.com,0.01


In [3]:
programmatic_query = """ 
SELECT
	geo_region,
	domain,
	toFloat64(sum(win_cost_micros_usd))/ 1000000 as cost,
	sum(clicks + companion_clicks) as clicks,
	count() as impressions,
	multiIf(clicks = 0, cost, cost / clicks) as CPC,
	today() - toDate(min(bid_time)) AS day_in_list
from
	Wins_buffer
where
	line_item_id in (113,109)
GROUP By
	geo_region,
	`domain`
having 
clicks > 0
and (impressions > 600 or day_in_list > 3)
order by
	CPC asc
"""


import pandas as pd
result, columns = client.execute(programmatic_query, with_column_types=True)
programmatic = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])


programmatic.drop(columns=['cost','clicks','impressions','CPC','day_in_list'])

a = programmatic.query('CPC < 0.5')
b = programmatic.query('CPC < 1 and CPC > 0.5')
c = programmatic.query('CPC > 1 and CPC < 1.5')
d = programmatic.query('CPC > 1.5 and CPC < 2')
e = programmatic.query('CPC > 2')


a['value'] = 1.5
b['value'] = 1.25
c['value'] = 0.7 
d['value'] = 0.35 
e['value'] = 0.01

# Higher_part = programmatic.query('CPC < 0.48')
# Lower_part = programmatic.query('CPC > 0.48 and CPC < 2')
# Dogshit_part = programmatic.query('CPC > 2')

# Higher_part['value'] = 1.3
# Dogshit_part['value'] = 0
# Lower_part['value'] = 0.7


C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [4]:
predict_pre = pd.concat([a,b,c,d,e,voidu_dogshit])

predict_pre

,geo_region,domain,cost,clicks,impressions,CPC,day_in_list,value
0,GBR/RIC,streetsofrogue.gamepedia.com,0.000236,1,1,0.000236,85,1.50
1,GBR/WSX,apester.com,0.000241,1,1,0.000241,58,1.50
2,FRA/50,m.coolrom.com.au,0.000338,1,1,0.000338,5,1.50
3,GBR/SND,fancyplanet.org,0.000349,1,1,0.000349,18,1.50
4,FRA/83,www.wot-life.com,0.000349,1,1,0.000349,37,1.50
5,CAN/BC,jeu.info,0.000370,1,1,0.000370,10,1.50
6,GBR/LEC,game-cdn.poki.com,0.000370,1,1,0.000370,47,1.50
7,USA/NJ,www.androidpit.com,0.000370,1,1,0.000370,8,1.50
8,GBR/BEN,paperio.site,0.000370,1,1,0.000370,69,1.50
9,GBR/BIR,www.vg247.com,0.000391,1,1,0.000391,4,1.50


In [5]:
predict = predict_pre.drop(columns = ['cost','clicks','impressions','CPC','day_in_list']) 
predict.to_csv("Kizi113_predict", sep ='|',encoding='utf-8', index= False )

In [6]:
import boto3

client = boto3.client('s3', 
                      region_name='us-east-1')

client.upload_file(r'C:\Users\izzet.metin\Dojo\Kizi113_predict',
                   'beeswax-data-us-east-1',
                   'bid_models/improvedigital/customer_data/Kizi113_predict.csv', ExtraArgs = {'ACL': 'bucket-owner-full-control'})

In [9]:
# Out of Bid Model Programmatic kizi
query = """ 
SELECT
	geo_region,
	domain,
	toFloat64(sum(win_cost_micros_usd))/ 1000000 as cost,
	sum(clicks + companion_clicks) as clicks,
	count() as impressions,
	multiIf(clicks = 0, cost, cost / clicks) as CPC,
	today() - toDate(min(bid_time)) AS day_in_list,
	model_id 
from
	Wins_buffer
WHERE
	line_item_id = 113 and geo_region like '' or `domain`  like '' 
GROUP by
`domain`,
model_id,
geo_region
having clicks > 0
and (impressions > 600 or day_in_list > 3)
order by CPC desc

""" 

result, columns = client.execute(query, with_column_types=True)

OOBM_programmatic = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

OOBM_programmatic.drop(columns=['cost','clicks','impressions','CPC','day_in_list','model_id'])
Higher_part_O = OOBM_programmatic.query('CPC < 0.48')
Lower_part_O = OOBM_programmatic.query('CPC > 0.48 and CPC < 2')
Dogshit_part_O = OOBM_programmatic.query('CPC > 2')

Higher_part_O['value'] = 1.3
Dogshit_part_O['value'] = 0
Lower_part_O['value'] = 0.7


C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [10]:
# OOBM Dogshit Kizi
query = """ 
SELECT
	sum(clicks + companion_clicks) as clicks,
	toFloat64(sum(win_cost_micros))/1000000  as CPC,
    geo_region,
	`domain`,
	count() as impressions,
	model_id,
	today() - toDate(min(bid_time)) AS day_in_list
from
	Wins_buffer
WHERE
	line_item_id = 113 and geo_region like '' or `domain`  like '' 
GROUP by
`domain`,
model_id,
geo_region
having 
clicks = 0 and CPC > 0.3
order by
	CPC desc
""" 

result, columns = client.execute(query, with_column_types=True)

OOBM_dogshit = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])
OOBM_dogshit['value'] = 0 
OOBM_dogshit.drop(columns=['clicks','impressions','CPC','day_in_list','model_id'])

,geo_region,domain,value
0,NLD/GR,,0
1,DEU/ST,,0
2,nw,,0
3,rm,,0
4,NL-GR,,0
5,ga,,0
6,FR-T,,0
7,ITA/MN,,0
8,ME,,0
9,FR-M,,0


In [11]:
predict_pre = pd.concat([Higher_part,Higher_part_O,Lower_part,Lower_part_O,Dogshit_part,Dogshit_part_O,voidu_dogshit,OOBM_dogshit])

predict_pre

C:\Users\izzet.metin\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,CPC,clicks,cost,day_in_list,domain,geo_region,impressions,model_id,value
0,0.000573,1,0.000573,4,ingame.de,eal,1,NaN,1.3
1,0.000594,1,0.000594,33,gta5-mods.com,eal,1,NaN,1.3
2,0.001197,1,0.001197,37,coolmathgames.com,RFW,3,NaN,1.3
3,0.001416,1,0.001416,7,www.gamekult.com,FRA/10,3,NaN,1.3
4,0.001543,1,0.001543,31,darkestdungeon.gamepedia.com,FRA/16,2,NaN,1.3
5,0.001738,1,0.001738,32,primagames.com,GBR/RFW,3,NaN,1.3
6,0.001908,1,0.001908,4,w3schools.com,GB-DNC,4,NaN,1.3
7,0.002134,5,0.010671,8,ninjakiwi.com,GBR/ABE,17,NaN,1.3
8,0.002363,1,0.002363,34,fnbr.co,GBR/DAL,4,NaN,1.3
9,0.002368,1,0.002368,27,www.egscomics.com,USA/UT,4,NaN,1.3


In [13]:
predict = predict_pre.drop(columns = ['cost','clicks','impressions','CPC','day_in_list'] ) 
predict.to_csv("Kizi113_predict", sep ='|',encoding='utf-8', index= False )

In [14]:
import boto3

client = boto3.client('s3', 
                      region_name='us-east-1')

client.upload_file(r'C:\Users\izzet.metin\Dojo\Kizi113_predict',
                   'beeswax-data-us-east-1',
                   'bid_models/improvedigital/customer_data/Kizi113_predict.csv', ExtraArgs = {'ACL': 'bucket-owner-full-control'})